In [ ]:
import random
import calendar
import time
import openpyxl
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
import base64
from base64 import b64encode, b64decode


def chash(s):
  hash = hashlib.sha256(s.encode()).hexdigest()
  return hash


class AESCipher(object):
  def __init__(self, key):
    self.block_size = AES.block_size
    self.key = hashlib.sha256(key.encode()).digest()

  def encrypt(self, plain_text):
    plain_text = self.__pad(plain_text)
    iv = Random.new().read(self.block_size)
    cipher = AES.new(self.key, AES.MODE_CBC, iv)
    encrypted_text = cipher.encrypt(plain_text.encode())
    return b64encode(iv + encrypted_text).decode("utf-8")

  def decrypt(self, encrypted_text):
    encrypted_text = b64decode(encrypted_text)
    iv = encrypted_text[:self.block_size]
    cipher = AES.new(self.key, AES.MODE_CBC, iv)
    plain_text = cipher.decrypt(encrypted_text[self.block_size:]).decode("utf-8")
    return self.__unpad(plain_text)

  def __pad(self, plain_text):
    number_of_bytes_to_pad = self.block_size - len(plain_text) % self.block_size
    ascii_string = chr(number_of_bytes_to_pad)
    padding_str = number_of_bytes_to_pad * ascii_string
    padded_plain_text = plain_text + padding_str
    return padded_plain_text

  @staticmethod
  def __unpad(plain_text):
    last_character = plain_text[len(plain_text) - 1:]
    return plain_text[:-ord(last_character)]


def bxor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])


def server_side_registration(id, rpw):

  # generate random secret y
  y = str (random.randrange(1000000, 9999999, 1))
 
	# get server secrets 
  file = open("server_secrets.txt", "r")
  a = file.readline()
  b = file.readline()
  ks = file.readline()
  a = a[:-1]
  b = b[:-1]

  aes_obj = AESCipher(ks)

  d_id = aes_obj.encrypt(id + y)

  temp = bxor(id.encode(), a.encode())
  alpha = chash(temp.decode() + b)

  temp = bxor(id.encode(), rpw.encode())
  temp = chash(temp.decode())
  beta = bxor(alpha.encode(), temp.encode())
  beta = beta.decode()

  temp = bxor(alpha.encode(), rpw.encode())
  temp = chash(temp.decode())
  gamma = bxor(y.encode(), temp.encode())
  gamma = gamma.decode()

  psi = chash(id + rpw + y + alpha)

  return [beta, gamma, d_id, psi]


def client_side_registration():

	# input id password for registration
  print ("--> Enter ID: ", end = '')
  id_ = input()
  print ("--> Enter Password: ", end = '')
  password_ = input()
  print ("--> Confirm Password: ", end = '')
  confirm_password_ = input()

	# check condition
  if password_ != confirm_password_ :
    print ("** Password & Confirm Password don't match! Please try again!")
    return

	# generate random secret m
  m = str (random.randrange(2874284, 764517642548164562, 1))

	# calculate rpw
  rpw = chash(str (m) + password_)

  params = server_side_registration(id_, rpw)

  wb_obj = openpyxl.load_workbook("users.xlsx")
  sheet_obj = wb_obj.active
  rows = sheet_obj.max_row
  ok = True
  for i in range(1, rows + 1):
    if sheet_obj.cell(row = i, column = 6).value == id_:
      ok = False
      break

  if ok == True:
    temp = chash(id_ + password_)
    eta = bxor(m.encode(), temp.encode()).decode()
    params.append(eta)
    for i in range(1, rows + 1):
      if sheet_obj.cell(row = i, column = 3).value == None:
        ind = int (1)
        for x in params:
          message_bytes = x.encode()
          base64_bytes = base64.b64encode(message_bytes)
          sheet_obj.cell(row = i, column = ind).value = base64_bytes.decode()
          ind += 1   
        sheet_obj.cell(row = i, column = ind).value = id_
        break
    wb_obj.save("users.xlsx")
    print ("** Succefully registered!")

  else:
    print ("** Username already exists!")


def server_side_auth(d_id, omega, theta, ts):
  gmt = time.gmtime()
  ts_server = str (calendar.timegm(gmt))
  if (int (ts_server) - int (ts)) > 1:
    print ("** Something went wrong!")
    return []

  # get server secrets 
  file = open("server_secrets.txt", "r")
  a = file.readline()
  b = file.readline()
  ks = file.readline()
  a = a[:-1]
  b = b[:-1]

  aes_obj = AESCipher(ks)

  extracted = aes_obj.decrypt(d_id)

  id = extracted[:-7]
  y = ""
  for i in range (len(extracted) - 7, len(extracted)):
    y += extracted[i]

  temp = bxor(id.encode(), a.encode()).decode() + b
  alpha = chash(temp)
  temp = bxor(alpha.encode(), y.encode()).decode()
  theta_n = chash(id + alpha + y + temp + ts)

  if theta != theta_n:
    print ("** Something went wrong!")
    return []

  mu = chash(id + y + temp + ts_server)

  return [mu, ts_server]

def client_side_auth():
  print ("--> Enter ID: ", end = '')
  id_ = input()
  print ("--> Enter Password: ", end = '')
  password_ = input()

  wb_obj = openpyxl.load_workbook("users.xlsx")
  sheet_obj = wb_obj.active
  rows = sheet_obj.max_row
  ok = False
  row_no = -1
  for i in range(1, rows + 1):
    if sheet_obj.cell(row = i, column = 6).value == id_:
      row_no = i
      ok = True
      break

  if ok == False:
    print ("** ID doesn't exist!")
    return

  beta = base64.b64decode(sheet_obj.cell(row = row_no, column = 1).value.encode()).decode()
  gamma = base64.b64decode(sheet_obj.cell(row = row_no, column = 2).value.encode()).decode()
  d_id = base64.b64decode(sheet_obj.cell(row = row_no, column = 3).value.encode()).decode()
  psi = base64.b64decode(sheet_obj.cell(row = row_no, column = 4).value.encode()).decode()
  eta = base64.b64decode(sheet_obj.cell(row = row_no, column = 5).value.encode()).decode()

  m = bxor(eta.encode(), chash(id_ + password_).encode()).decode()
  rpw = chash(m + password_)
  alpha = bxor(beta.encode(), chash(bxor(id_.encode(), rpw.encode()).decode()).encode()).decode()
  y = bxor(gamma.encode(), chash(bxor(alpha.encode(), rpw.encode()).decode()).encode()).decode()
  psi_n = chash(id_ + rpw + y + alpha)

  if psi_n != psi:
    print ("** Something went wrong!")
    return
  
  gmt = time.gmtime()
  ts = str (calendar.timegm(gmt))

  temp = bxor(id_.encode(), alpha.encode()).decode()
  omega = bxor(y.encode(), chash(temp).encode()).decode()
  temp = bxor(temp.encode(), ts.encode()).decode()
  omega = bxor(omega.encode(), chash(temp).encode()).decode()

  temp = bxor(alpha.encode(), y.encode()).decode()
  theta = chash(id_ + alpha + y + temp + ts)

  params = server_side_auth(d_id, omega, theta, ts)

  if (len(params) == 0):
    return
  
  gmt = time.gmtime()
  n_ts = str (calendar.timegm(gmt))

  if (int (n_ts) - int (params[1])) > 1:
    print ("** Something went wrong!")
    return

  temp = bxor(alpha.encode(), y.encode()).decode()
  mu = chash(id_ + y + temp + params[1])

  if mu != params[0]:
    print ("** Something went wrong!")
    return

  temp = bxor(id_.encode(), alpha.encode())
  temp = bxor(temp, y.encode())
  temp = bxor(temp, ts.encode())
  temp = bxor(temp, params[1].encode())
  temp = temp.decode()
  shared_sk = chash(temp)

  message_bytes = shared_sk.encode()
  base64_bytes = base64.b64encode(message_bytes)
  sheet_obj.cell(row = row_no, column = 7).value = base64_bytes.decode()
  wb_obj.save("users.xlsx")

  print ("* Successfully Logged in!")

client_side_registration()
client_side_auth()

--> Enter ID: 1
--> Enter Password: 22
* Successfully Logged in!
